In [1]:
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
from src import DataModule, Model
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

In [23]:
config = {
    'lr': 3e-4,
    'optimizer': 'Adam', # new
    'batch_size': 64,   
    'max_epochs': 50,
    'precision': 16,
    'test_size': 0.2
    'seed': 42,
    'subset': 0.1,
    'size': 256
    'backbone': 'resnet18'
    'val_batches': 10
}

dm = DataModule(**config)

model = Resnet(config)

wandb_logger = WandbLogger(project="cassava", config=config)      # Logging with weights and biases

es = EarlyStopping(monitor='val_acc', mode='max', patience=3)
checkpoint = ModelCheckpoint(
    dirpath='./',
    filename='{config["backbone"]}-{config["size"]}-{val_acc:.5f}', 
    save_top_k=1, monitor='val_acc', mode='max')

trainer = pl.Trainer(
    gpus=1, 
    precision=config['precision'],
    logger=wandb_logger,
    max_epochs=config['epochs'],
    #limit_train_batches=5,
    #limit_val_batches=5
    callbacks=[es, checkpoint],
    limit_val_batches=config['val_batches']
)

trainer.fit(model, dm)


GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Using native 16bit precision.


Training samples:  17117
Validation samples:  4280
Training with  343 samples.



  | Name   | Type       | Params
--------------------------------------
0 | resnet | ResNet     | 11.2 M
1 | trans  | Sequential | 0     
--------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params


RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 1.96 GiB total capacity; 611.37 MiB already allocated; 15.12 MiB free; 614.00 MiB reserved in total by PyTorch)